In [2]:
import numpy as np

# A four months American put with strike $50 on an asset with spot price $48 paying dividends
# continuously at rate 1% is worth $4.08. The risk free interest rates are constant at 2%
# parameters
S0 = 48
K = 50
r = 0.02
q = 0.01
T = 4/12
N = 10
tol = 1e-6

# Identify the number of time steps Nfixed in the trinomial tree pricer which gives the value
# of a four months put option within 10^-6 tolerance:
# Start with 10 time steps for the given option and double the time steps until two consecutive
# approximations are within tolerance of each other. The finest tree considered corresponds to
# Nfixed time steps.

def european_put_trinomial(S, K, T, r, q, sigma, N):
    dt = T / N
    u = np.exp(sigma * np.sqrt(3*dt))
    d = 1 / u
    pu = 1/6 + (r-q-sigma**2*0.5) * np.sqrt(dt/(12*sigma**2))
    pm = 2/3
    pd = 1/6 - (r-q-sigma**2*0.5) * np.sqrt(dt/(12*sigma**2))

    # Build the binomial tree
    option_tree = np.zeros((N + 1, 2*N + 1))
    for j in range(2*N + 1):
        option_tree[N][j] = max(0, K - S * (u ** (N - j)))

    # Backward induction
    for i in range(N - 1, -1, -1):
        for j in range(2*i+1):
            option_tree[i][j] = np.exp(-r * dt) * (pu * option_tree[i + 1][j] + pm * option_tree[i + 1][j + 1] + pd * option_tree[i+1][j+2])

    return option_tree[0][0]


# american option
def american_put_trinomial(S, K, T, r, q, sigma, N):
    dt = T / N
    u = np.exp(sigma * np.sqrt(3*dt))
    d = 1 / u
    pu = 1/6 + (r-q-sigma**2*0.5) * np.sqrt(dt/(12*sigma**2))
    pm = 2/3
    pd = 1/6 - (r-q-sigma**2*0.5) * np.sqrt(dt/(12*sigma**2))

    # Build the binomial tree for the option values
    option_tree = np.zeros((N + 1, 2*N + 1))
    for j in range(2*N + 1):
        option_tree[N][j] = max(0, K - S * (u ** (N - j)))

    # Backward induction
    for i in range(N - 1, -1, -1):
        for j in range(2*i+1):
            intrinsic = K - S * (u ** (i - j))
            option_tree[i][j] = max(intrinsic, np.exp(-r * dt) * (pu * option_tree[i + 1][j] + pm * option_tree[i + 1][j + 1] + pd * option_tree[i+1][j+2]))

    return option_tree[0][0]

sigma = 0.272903
N = 10
prev = 0
while True:
    V = american_put_trinomial(S0, K, T, r, q, sigma, N)
    if abs(V - prev) < tol:
        break

    print('The number of time steps in the finest tree is', N)
    print('The option price is', V)

    prev = V
    N *= 2

# final iteration of the binomial tree pricer
N_fixed = N

print('The number of time steps in the finest tree is', N_fixed)
print('The option price is', V)


The number of time steps in the finest tree is 10
The option price is 4.113922966903458
The number of time steps in the finest tree is 20
The option price is 4.090449112674083
The number of time steps in the finest tree is 40
The option price is 4.0666561081447705
The number of time steps in the finest tree is 80
The option price is 4.082712243815011
The number of time steps in the finest tree is 160
The option price is 4.077886083345847
The number of time steps in the finest tree is 320
The option price is 4.08057621218413
The number of time steps in the finest tree is 640
The option price is 4.079957275290515
The number of time steps in the finest tree is 1280
The option price is 4.080178500668171
The number of time steps in the finest tree is 2560
The option price is 4.080117037374746


KeyboardInterrupt: 

In [5]:
import pandas as pd
import warnings 
warnings.filterwarnings('ignore')

S0 = 48
K = 50
r = 0.02
q = 0.01
T = 4/12
N = 10
tol = 1e-6

# american option
def american_put_trinomial(S, K, T, r, q, sigma, N):
    dt = T / N
    u = np.exp(sigma * np.sqrt(3*dt))
    d = 1 / u
    pu = 1/6 + (r-q-sigma**2*0.5) * np.sqrt(dt/(12*sigma**2))
    pm = 2/3
    pd = 1/6 - (r-q-sigma**2*0.5) * np.sqrt(dt/(12*sigma**2))

    # Build the binomial tree for the option values
    option_tree = np.zeros((N + 1, 2*N + 1))
    for j in range(2*N + 1):
        option_tree[N][j] = max(0, K - S * (u ** (N - j)))

    # Backward induction
    for i in range(N - 1, -1, -1):
        for j in range(2*i+1):
            intrinsic = K - S * (u ** (i - j))
            option_tree[i][j] = max(intrinsic, np.exp(-r * dt) * (pu * option_tree[i + 1][j] + pm * option_tree[i + 1][j + 1] + pd * option_tree[i+1][j+2]))

    return option_tree[0][0]

option_price = 4.08

sigma0 = 0.05  # Initial volatility
sigma1 = 1  # Initial volatility
prev = 0  # Previous option price

# Use the secant method with initial volatility approximations 5% volatility and 100% volatility, i.e., sigma0 = 0.05 and sigma1 = 1. Use tolerance 10−4
# . Report all the implied volatility approximations from the secant method recursion as well as the corresponding trinomial tree approximate
# values of the option

res = pd.DataFrame(columns=['sigma', 'option_price'])
N = 2560
while abs(sigma1 - sigma0) > tol:
    V = american_put_trinomial(S0, K, T, r, q, sigma1, N)
    print(sigma1, V)
    res = res.append({'sigma': sigma, 'option_price': V}, ignore_index=True)
    sigma = sigma1 - (sigma1-sigma0)*(V-option_price)/(V-prev)
    sigma0 = sigma1
    sigma1 = sigma
    prev = V
    
print('The implied volatility is', sigma)
print('The option price is', V)
res

1 12.031161907994044
0.37216339781984087 5.165674754989589
0.27288037040245666 4.079871171874831
0.2728921501078124 4.079999155759395
The implied volatility is 0.2728922278121704
The option price is 4.079999155759395


,sigma,option_price
0,0.272892,12.031162
1,0.372163,5.165675
2,0.27288,4.079871
3,0.272892,4.079999
